In [18]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import copy



In [19]:
f= open("parameters.txt","r")
n= int(f.readline())
print(n)
transitionMatrix= np.empty([n,n])
for  i in range(n):
    str = f.readline()
    l=[np.double(p) for p in str.split()]
    transitionMatrix[i] = l
# transitionMatrix= np.asmatrix(transitionMatrix)

Mean= f.readline()
print( "mean is ",Mean)
Mean=[np.double(p) for p in Mean.split()]
Variance = f.readline()
Variance=[np.double(p) for p in Variance.split()]
transitionMatrix

2
mean is  200	100



array([[0.7, 0.3],
       [0.1, 0.9]])

In [20]:
def initialEstimator(mat):
    mat1= copy.deepcopy(mat)
    print(mat1)
    # print(mat2)
    # print(x)
    for i in range(20):
    # while not(np.isclose(mat1, mat1[0]).all()):
        mat1= mat1 @ mat1
    # print(np.isclose(mat1, mat1[0]).all())
    print(mat1)
    return mat1[0]


initial= initialEstimator(transitionMatrix)



[[0.7 0.3]
 [0.1 0.9]]
[[0.25 0.75]
 [0.25 0.75]]


In [29]:
dataFile = open("data.txt","r")
data= np.double(dataFile.readlines())

dataSize = len(data)
emissionMatrix=np.empty((n,dataSize))
for i in range(n):
    emissionMatrix[i]= norm(Mean[i],np.sqrt(Variance[i])).pdf(data)

emissionMatrix=np.asmatrix(emissionMatrix).T
emissionMatrix.shape
np.savetxt("emissionMatrix.txt",emissionMatrix,delimiter=' ')


In [82]:
dpTable= np.zeros([dataSize,n]) #dpTable size 2x1000
backTrack = np.zeros([dataSize,n])
for i in range(n):
    # print(  initial[i ] ,  emissionMatrix[i,0])
    dpTable[0,i]=np.log(initial[i]) + np.log( emissionMatrix[0,i])
# temp1 = dpTable[0,0] + np.log(emissionMatrix[1,0]) + np.log(transitionMatrix[0,0])
# temp2 = dpTable[0,1] + np.log(emissionMatrix[1,0]) + np.log(transitionMatrix[1,0])
# print(temp1 ,temp2)


for i in range(1,dataSize):
    for  s in range (n):
        max=-np.Infinity
        prev=-1
        for k in range(n):
            temp= dpTable[i-1,k] + np.log(emissionMatrix[i,s]) + np.log(transitionMatrix[k,s])
            if ( temp > max) : 
                max = temp
                prev= k        
        dpTable[i][s] = max
        backTrack[i][s]= int(prev)

print(dpTable[dataSize-1])
ans =[]
ansIndex = int(np.argmax(np.array(dpTable[dataSize-1])))
ans.append(ansIndex)
for  i in range(dataSize-2,0,-1):
    # print(i ,ansIndex)
    ansIndex= backTrack[i,int(ansIndex)]
    ans.append(ansIndex)

ans= np.array(ans)
ans =np.flip(ans)
# print("after flip" ,ans)
ans= ans.tolist()

# print(ans)
for i in range(len(ans)):
    if ans[i]==0:
        ans[i]="El Nino"
    else :
        ans[i]= "La Nina"
np.savetxt("dpTable.txt",dpTable,delimiter=' ')
# np.savetxt("vitebriOutput.txt",ans,delimiter="\n")
with open('vitebriOutput.txt', 'w') as f:
    for item in ans:
        f.write("\"%s\"\n" % item)

        



[-68295.49751944 -67780.52661432]


In [58]:
ansIndex

1.0